In [58]:
from rtt_tools import dump_data
from rtt_tools.dump_data import *  # pussy died because of this wildcard import
from rtt_tools.export import *

logger = logging.getLogger(__name__)
coloredlogs.CHROOT_FILES = []
coloredlogs.install(level=logging.INFO, use_chroot=False)

In [59]:
from rtt_tools.gen.max_rounds import FUNC_DB, FuncDb, FuncInfo

In [4]:
import json
js1 = json.load(open('/tmp/rtt-results-full9.json'))

In [48]:
js2 = json.load(open('/tmp/rtt-results-full9-ext01.json'))

In [49]:
js = js1 + js2

In [50]:
#print(json.dumps(js[2829322], indent=2))
#for rix, rec in enumerate(js):
#    if 'lowmc' in rec['data_type']:
#        print(rec['data_type'])
#     exp_info = loader.break_exp_ph4(rec['data_type'])
#     if not exp_info:
#         print('Could not parse %s' % rec['data_type'])
#         continue
#len(js2)

In [60]:
loader = dump_data.Loader()

def exp_info_to_rec(exp_info):
    return collections.OrderedDict([
        ('tp', exp_info.fnc_type),
        ('f', exp_info.fnc_name),
        ('s', exp_info.size),
        ('m', exp_info.meth),
        ('r', exp_info.fnc_round),
        ('e', exp_info.id),
    ])

def name_to_dict(name):  # js[0]['data_type']
    exp_info = loader.break_exp_ph4(name)
    if exp_info is None or exp_info.fnc is None:
        exp_info = loader.break_exp_ph4_mpc(name)
    if exp_info is None or exp_info.fnc is None:
        return None
    return exp_info_to_rec(exp_info)


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

funtype = {}  # fun -> type
funrounds = collections.defaultdict(lambda: set())  # fun -> set of rounds tested
methsize = {}
testvariant = {}
datatype = {}
fnames = set()
test_db = collections.defaultdict(lambda: 0)
test_db_inv = collections.defaultdict(lambda: 0)
test_db_ctr = set()
tstart = time.time()

# func_name -> (meth, size) -> round -> test:var:subtest -> [pass, fail, fail]  # 3 seed results
funcres = collections.defaultdict(                   # fncname
    lambda: collections.defaultdict(                 # meth-size
        lambda: collections.defaultdict(             # round
            lambda: collections.defaultdict(         # test-variant-subtest
                lambda: collections.OrderedDict([        
                    ('info', None),
                    ('recidx', None),
                    ('exid', None),
                    ('exs', [None]*3),
                    ('exs_info', [None]*3),
])))))


for rix, rec in enumerate(js):
    data_type = rec['data_type']
    data_type = data_type.replace('stream:cipher', 'stream_cipher')
    exp_info = loader.break_exp_ph4(data_type)
    if exp_info is None or exp_info.fnc is None:
        exp_info = loader.break_exp_ph4_mpc(data_type)
    if not exp_info or not exp_info.fnc:
        print('Could not parse %s' % data_type)
        continue
    #if rix > 1000:
    #    break
        
    funtype[exp_info.fnc_name] = exp_info.fnc_type
    funrounds[exp_info.fnc_name].add(exp_info.fnc_round)

    meth_size = '%s:%s' % (exp_info.meth, exp_info.size)
    if 'testmpc' in data_type:
        meth_size = '%s:%s:%s' % (exp_info.meth, exp_info.spread, exp_info.size)
    if 't:prng-' in data_type:
        meth_size = '%s:b%s:%s' % (exp_info.meth, exp_info.fnc_block, exp_info.size)
    
    methsize[meth_size] = [exp_info.meth, exp_info.size, exp_info.spread]
    cround = exp_info.fnc_round  # TODO: not always int
    datatype[data_type] = exp_info_to_rec(exp_info)
    
    test_desc = f'{rec["test"]}##[{rec["variant_type"]}][{rec["variant"]}]##[{rec["subtest_type"]}][{rec["subtest"]}]'
    testvariant[test_desc] = collections.OrderedDict([        
        ('variant_type', rec["variant_type"]),
        ('variant', rec["variant"]),
        ('variant_id', rec["variant_id"]),
        ('subtest_type', rec["subtest_type"]),
        ('subtest', rec["subtest"]),
        ('exid', rec["exid"]),
    ])
    
    test_db_ctr.add(test_desc)
    fncresrec = funcres[exp_info.fnc_name][meth_size][cround][test_desc]
    fncresrec['exid'] = rec['exid']
    fncresrec['recidx'] = rix
    fncresrec['info'] = data_type
    fncresrec['einfo'] = exp_info
    
    # iterate over subtests
    nsubs_passed = 0
    for sub in rec['subs']:
        is_passed = sub['stats'][0]['pass']
        nsubs_passed += int(is_passed)
        
    nsubs = len(rec['subs'])
    npass_ratio = nsubs_passed / nsubs
    variant_passed = npass_ratio >= 0.99  # tune this ratio! Ratio of passed subtests to consider test passed
    
    if exp_info.id >= 3:
        print(f'!! Out of range {exp_info.id} for {data_type}')
        continue
        
    fncresrec['exs'][exp_info.id] = variant_passed
    fncresrec['exs_info'][exp_info.id] = collections.OrderedDict([
        ('exid', rec['exid']),
        ('recidx', rix),
        ('info', data_type),
        ('einfo', exp_info),
        ('nsubs', nsubs),
        ('nsubsp', nsubs_passed),
        ('nsubsr', npass_ratio),
    ])
    # print('#: %s, pass: %s, pct: %2.7f' % (nsubs, nsubs_passed, 100*nsubs_passed/nsubs))
    
# Reindex tests so we have stable IDs
all_tests = sorted(list(test_db_ctr))
for ix, test in enumerate(all_tests):
    test_db[test] = ix
    test_db_inv[ix] = test
    
#print(funrounds, funtype, funcres)
#print(json.dumps(funcres, indent=2))
print('Proc1 done in %s s' % (time.time() - tstart))

In [ ]:
# funcres["AES"]["ctr:10485760"][3]
# Analyze function -> meth-size -> round; all tests, reject if 2/3 rejected
tstart = time.time()

# All experiments computed
data_type_set = set()

# func_name -> (meth, size) -> round -> {summary dict}
funcres_agg = collections.defaultdict(               # fncname
    lambda: collections.defaultdict(                 # meth-size
        lambda: collections.defaultdict(             # round        
            lambda: collections.OrderedDict([        
                ('num_tests', 0),
                ('num_rej', 0),
                ('rejects', False),
                ('lib_num_tests', 0),
                ('lib_num_rej', 0),
                ('lib_rejects', False),
                ('info', None),
                ('einfo', None),
                ('exid', None),
                ('exs_info', collections.defaultdict(lambda: collections.OrderedDict([
                    ('exids', set()),
                    ('num_tests', 0),
                    ('num_rej', 0),
                    ('lib_rejects', 0),
                    ('rejs', set()),
                    ('all_tests', set()),
                ]))),
])))) 

# Liberal thresholding - both of the above
liberal_threshold_ratio = 0.011  # at least 0.011 rejection rate
liberal_threshold_const = 3  # at least 3 tests
conservative_ratio = 0.01  # num_rej / num_tests; num_rej += 1 if the same tests was rejected at least twice; # TUNE THIS!

ignore_random_excursions = True
ignore_testmpc = False

uncomps = 0
uncomps_funcs = collections.Counter()
uncomps_set = set()
uncomps_bat = set()
rerun_cmds = collections.defaultdict(lambda: set())
nonames_set = set()
rerun_tests = collections.defaultdict(lambda: collections.defaultdict(lambda: None))  # ename -> test -> aux data

bat_map = {
    'NIST Statistical Testing Suite': '--nist_sts',
    'Dieharder': '--dieharder',
    'TestU01 Alphabit': '--tu01_alphabit',
    'TestU01 Small Crush': '--tu01_crush',
    'TestU01 Rabbit': '--tu01_rabbit',
    'TestU01 Block Alphabit': '--tu01_blockalphabit',
    'TestU01 Crush': '--tu01_crush',
    'booltest_1': '--booltest-1',
    'booltest_2': '--booltest-2',
}

expfiles = []
expfiles_path = '../ph4-01/exps-dump/__expfiles.json'
if os.path.exists(expfiles_path):
    with open(expfiles_path) as fh:
        expfiles = json.load(fh)
        
        
def find_expfile(tpl, expfiles):
    for fname in expfiles:
        if tpl in fname:
            yield fname

            
for fname in funcres:
    for meth_size in funcres[fname]:
        for cround in funcres[fname][meth_size]:
            ctests = funcres[fname][meth_size][cround]  # all tests map
            crec = funcres_agg[fname][meth_size][cround]  # resulting dict
            
            for ctest in ctests:
                data_type_set.add(ctests[ctest]["info"])
                
                for resix, resex in enumerate(ctests[ctest]['exs']):
                    if resex is None:
                        uncomps += 1
                        uncomps_funcs[fname] += 1
                        ename = ctests[ctest]["info"]
                        
                        if ignore_random_excursions and 'Random Excursions' in ctest:
                            continue
                        if ignore_testmpc and 'testmpc' in ename:
                            continue
                            
                        if '-e:' in ename:
                            ename2 = re.sub(r'-e:[\d]-', f'-e:{resix}-', ename)  # get seed run we need
                            ename2 = re.sub(r'-i:([\w]+(:?\.key)?)-.*', '-i:\\1', ename2)  # cut off input specifications
                            ename2 = re.sub(r'^PH4-SM-[\d]+-', '', ename2)  # remove testing prefix, anything goes
                            fnames = list(find_expfile('-' + ename2.replace(':', '_'), expfiles))
                            if len(fnames) == 0:
                                # print(f'! no fnames found. Cfg: {fname} {meth_size} {cround} {ctest} | ename: {ename} | resix: {resix}. (Check expfiles.json)')
                                nonames_set.add(ename)
                                
                            if len(fnames) != 1:
                                # print(f'- more fnames than 1 ({len(fnames)}): {fnames}, Cfg: {fname} {meth_size} {cround} {ctest} | ename: {ename} | resix: {resix}')
                                fnames.sort(key=natural_sort_key)
                                fnames = list(reversed(fnames))
                                
                            ename = fnames[0] if fnames else ename
                            ename = ename.replace('.json', '')
                            ename = ename.replace('_', ':')
                            ename = ename.replace('stream:cipher', 'stream_cipher')

                        elif 'testmpc' in ename:
                            ename = re.sub(r'-inp-([\w]+?)([\d]+)-', '-inp-\\1!XXXX', ename)
                            ename = ename.replace('!XXXX', '%02d-' % resix)
                            ename = ename.replace('.json', '')
                            ename = re.sub(r'^.*-testmpc', 'testmpc', ename)
                        
                        uncomps_set.add(ename)
                        uncomps_bat.add(ctest.split('|')[0])
                        rerun_cmds[ename].add(ctest.split('|')[0])
                        rerun_tests[ename][ctest] = (cround, ctests[ctest]["exs"], ctests[ctest]["recidx"], ctests[ctest]["info"])
                        # print(f'Uncomputed: {fname}-{meth_size}-{cround}-{ctest}-{ctests[ctest]["exs"]}-{ctests[ctest]["recidx"]}, ename: {ename}, orig: {ctests[ctest]["info"]}')
                    
                if None in ctests[ctest]['exs']:                    
                    # print(json.dumps(js[ctests[ctest]["recidx"]], indent=2))
                    continue
                    
                # Liberal interpretation: threshold of tests is enough to tell that we reject the generator.
                crec['lib_num_tests'] += 1
                
                # Iterating over seed variant runs for given fname-methsize-round-ctest
                exs_info = ctests[ctest]['exs_info']
                for ix, exsinfo in enumerate(exs_info):
                    if exsinfo is None:
                        continue  # test was not found for this exp.id (seed alternation)
                        
                    # Single test rejects single seed stream
                    is_rejected = int(not ctests[ctest]['exs'][ix]) if ctests[ctest]['exs'][ix] is not None else 0 
                    ecrec = crec['exs_info'][ix]  # ecrec ~ funcres_agg[fname][meth_size][cround]['exs_info'][ix]
                    ecrec['exids'].add(ctests[ctest]['exs_info'][ix]['exid'])
                    ecrec['num_tests'] += 1
                    ecrec['num_rej'] += is_rejected
                    ecrec['all_tests'].add(test_db[ctest])
                    if is_rejected:
                        ecrec['rejs'].add(test_db[ctest])
                        
                    # With each update recompute lib_rejects criteria based on the current test results.    
                    ecrec['lib_rejects'] = ecrec['num_rej'] >= liberal_threshold_const and (ecrec['num_rej'] / ecrec['num_tests']) >= liberal_threshold_ratio
                    
                    # Global liberal view, summarizind seed runs
                    crec['lib_num_tests'] += 1
                    crec['lib_num_rej'] += is_rejected
                    
                # Conservative interpretation: We reject if we can reproduce the *same test* at least 2 times. 
                # 'num_rej' is then number of tests that rejected the same input at least 2 times. 
                is_rejected = sum(ctests[ctest]['exs']) <= 1  # counting passes, 1 or less passes -> rejected (2+ rejections)
                crec['num_tests'] += 1  # +1 for each test computed for fnc-methsize-round
                crec['num_rej'] += int(is_rejected)  # +1 for each rejection by a single test for fnc-methsize-round, reproduced on 3 buckets
                crec['info'] = ctests[ctest]['info']
                crec['einfo'] = ctests[ctest]['einfo']
                crec['exid'] = ctests[ctest]['exid']

            if crec['num_tests'] == 0:
                print(f'! zero tests: f:{fname} m:{meth_size} r:{cround}')
                continue
                
            cratio = crec['num_rej'] / crec['num_tests']
            crec['rejects'] = cratio >= conservative_ratio
            crec['lib_rejects'] = sum([int(crec['exs_info'][kk]['lib_rejects']) for kk in crec['exs_info'].keys()]) > 1 # 2 or more rejects

print('Proc2 done in %s s' % (time.time() - tstart))

In [ ]:
print('* Unrun Experiments:')
kspaces = [20, 38, 20, 20, 20, 20, 20]
krec = re.compile(r'^\[(.*?)\]\s*\[(.*?)\]$')

for ename in []:#sorted(list(rerun_tests.keys())):
    recs = sorted(list(rerun_tests[ename].keys()))
    print(f'\n{ename}')
    for k in recs:
        rec = rerun_tests[ename][k]
        kcomp = k.split("|", 1)
        kcomp = [kcomp[0]] + kcomp[1].split('##')
        kcompr = kcomp[:2]
        for ixx, cxx in enumerate(kcomp[2:]):
            m = krec.match(cxx)
            if not m:
                kcompr.append(cxx)
                continue
            kcompr.append(m.group(1))
            kcompr.append(m.group(2))
            
        kform = ' | '.join([x.ljust(kspaces[ix] if ix < len(kspaces) else 20) for ix, x in enumerate(kcompr)])
        print(f'  {"%21s" % rec[1]} | {kform} | {rec[2]} {rec[3]}')
    #raise ValueError()
            
print('\n\n\n\n\n\n\n--------------------\nUncomputed total:', uncomps, 'functions:', uncomps_funcs, ' batteries:', uncomps_bat)
for ename in sorted(list(rerun_cmds.keys())):
    fname = ename.replace(':', '_')
    fname += '.json'
    bats = sorted(list(rerun_cmds[ename]))
    ename2 = ename.replace('-SM-01-', '-SM-04-')
    ename2 = ename2.replace('-SM-02-', '-SM-04-')
    fsize = ('100' if '100MiB' in fname else '10') if 'MiB' in fname else ('100' if '100MB' in fname else '10')
    print(f"submit_experiment {' '.join([bat_map[x] for x in bats])} --name '{ename2}' --cfg '/home/debian/rtt-home/RTTWebInterface/media/predefined_configurations/{fsize}MB.json' --cryptostreams-config '{fname}'")
    
print('\n\n\n-------------------Resubmit summary:')
for ename in sorted(list(rerun_cmds.keys())):
    bats = sorted(list(rerun_cmds[ename]))
    fsize = ('100' if '100MiB' in fname else '10') if 'MiB' in fname else ('100' if '100MB' in fname else '10')
    print(f"{ename} : {' '.join([bat_map[x] for x in bats])} @ {fsize}MB")

print('\n\n\n-------------------No names sum:')
for x in sorted(list(nonames_set)):
    print(x)
    
print('-----------------\n\n\nProc2 done') 
print('Num of computed experiments: %s' % (len(data_type_set)))


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Liberal thresholding
# TODO: compute higher rounds for liberal interpretation, e.g., Tangle has not enough
use_liberal = True
funcs_no_rounds = set(['MCSSHA3', 'Shabal'])

log_lines = []
table_lines = []
tests_to_print = set()

exids_recomp_margin = 3  # rounds a top of maximal / the latest fail; only 10 & 100 MB (faster)
exids_recompute = collections.defaultdict(  # fname
    lambda: collections.defaultdict(        # meth:size
        lambda: [[],[]]))                   # set of rounds to compute

exids_recomp_margin_1000MB = 1  # rounds a top of maximal / the latest fail
exids_recompute_1000MB = collections.defaultdict(  # fname
    lambda: collections.defaultdict(        # meth:size
        lambda: None))                      # set of rounds to compute

exids_computed_1000MB = collections.defaultdict(  # fname
    lambda: collections.defaultdict(              # meth:size
        lambda: 0))

compute_cols = collections.defaultdict(     # ftype:fname
    lambda: collections.defaultdict(        # meth:size
        lambda: list()))                    # set of rounds to compute

computed_cols = collections.defaultdict(     # ftype:fname
    lambda: collections.defaultdict(         # meth:size
        lambda: set()))                      # set of rounds to compute


def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
    return [int(text) if text.isdigit() else text.lower()
            for text in _nsre.split(s)]


for fname in sorted(funcres_agg.keys(), key=natural_sort_key):
    fname_low = fname.lower()
    
    # Find info per-fname
    fmeth = list(funcres_agg[fname].keys())
    fround = list(funcres_agg[fname][fmeth[0]].keys()) if fmeth else None
    finfo = funcres_agg[fname][fmeth[0]][fround[0]] if fround else None
    finfo = finfo['einfo'] if finfo and 'einfo' in finfo else None
        
    erec = FUNC_DB.search(fname, FuncInfo.HASH if finfo is not None and finfo.fnc_type == 'hash' else FuncInfo.CIPHER)
    if erec is None and finfo.fnc_type == 'prng':
        erec = FuncInfo(fname, FuncInfo.PRNG, 0, 1, 1)
    
    max_rounds_cur = erec.max_rounds if erec else None
    crypt_rounds_cur = erec.human_broken_rounds if erec else None
    max_rounds_cur_s = ('%3d' % max_rounds_cur) if max_rounds_cur is not None else ' - '
    crypt_rounds_cur_s = ('%3d' % crypt_rounds_cur) if crypt_rounds_cur is not None else ' - '
    max_rnd_broken = None
    max_rnd_broken_100 = None
    max_rnd_broken_meth = collections.defaultdict(lambda: 0)
    max_rnd_broken_meth_100 = collections.defaultdict(lambda: 0)
    min_rnd_meth = collections.defaultdict(lambda: 999)
    max_rnd_meth = collections.defaultdict(lambda: 0)
    comp_rounds_meth = collections.defaultdict(lambda: list())
    ftypename = f'{erec.ftype}:{fname}'
    
    for meth in sorted(funcres_agg[fname].keys()):
        namemeth = funcres_agg[fname][meth]
        #if '104857600' not in meth: # or 'sac:' in meth:
        #    continue
        
        h100MB = 104857600
        h1000MB = 1048576000
        meth_parts = meth.split(':')
        meth_size = int(meth_parts[-1])
        is_below_1000 = meth_size < h1000MB
        is_col_type = '.key' in meth
        
        rounds = list(namemeth.keys())
        for ix, rnd in enumerate(rounds):
            if rnd == '1-0-0':
                rounds[ix] = 1
                namemeth[1] = namemeth['1-0-0']
            if rnd == 'x':
                namemeth[1] = namemeth['x']
                rounds[ix] = 1
            if rnd is None:
                namemeth[1] = namemeth[None]
                rounds[ix] = 1
                
        rounds.sort()
        cexids = [-1]*3
        min_rnd_meth[meth] = rounds[0]
        max_rnd_meth[meth] = rounds[-1]
        comp_rounds_meth[meth] = rounds
        for rnd in rounds:
            crec = namemeth[rnd]
            cexid = crec['exid']
            
            for ixx in crec['exs_info'].keys():
                cexids[ixx] = max(cexids[ixx], max(crec['exs_info'][ixx]['exids']))
            
            cur_rejects = crec['lib_rejects'] if use_liberal else crec['rejects']
            cnum_rej = crec["lib_num_rej"] if use_liberal else crec["num_rej"]
            cnum_tests = crec["lib_num_tests"] if use_liberal else crec["num_tests"]
            
            con_rejects = crec['rejects']
            con_num_rej = crec["num_rej"]
            con_num_tests = crec["num_tests"]
            
            # Conservative view on rejection - same test fails at least twice on 3 seed streams
            # Liberal: X tests per seed run. Needs to be set properly.
            if cur_rejects and (max_rnd_broken is None or rnd > max_rnd_broken):
                max_rnd_broken = rnd
                
            if cur_rejects and is_below_1000 and (max_rnd_broken_100 is None or rnd > max_rnd_broken_100):
                max_rnd_broken_100 = rnd
                    
            if cur_rejects and (max_rnd_broken_meth[meth] is None or rnd > max_rnd_broken_meth[meth]):
                max_rnd_broken_meth[meth] = rnd
            
            if cur_rejects and is_below_1000 and (max_rnd_broken_meth_100[meth] is None or rnd > max_rnd_broken_meth_100[meth]):
                max_rnd_broken_meth_100[meth] = rnd
            
            if is_col_type and is_below_1000:
                computed_cols[ftypename][meth].add(rnd)
                
            # Detailed view on which tests failed in which experiments
            # In conservative interpretation, we require the same test to fail at least twice / 3 runs.
            # 'num_rej' thus does not have to match rejected tests in particular seed run 
            # (e.g., if test IDs are disjunct, 'num_rej' is 0)
            rejs_details = ''
            if cnum_rej < 15:  # disabled for brevity
                rejs_details = []
                for kix in crec['exs_info'].keys():
                    exs_info2 = crec['exs_info'][kix]
                    rejs_details.append(sorted(list(exs_info2['rejs'])))
                rejs_details = "[" + "; ".join([", ".join(['%3d' % y for y in x]) for x in rejs_details]) + "]"
            
            conv_str = ''
            if use_liberal:
                conv_str = '; c %s (%3d/%3d)' % ('R' if con_rejects else '-', con_num_rej, con_num_tests)
            
            margin_s = ('margin: %.2f %%' % (100.0 * (1 - rnd/max_rounds_cur))) if cur_rejects and max_rounds_cur is not None else '' 
            vdict = '%s (%3d/%3d%s | %s %s max. %s) %s' % ('R' if cur_rejects else '-', cnum_rej, cnum_tests, conv_str, crypt_rounds_cur_s, max_rounds_cur_s, margin_s, rejs_details)
            irnd = rnd
            try:
                irnd = int(rnd)
            except:
                pass
            srnd = ('%4d' if isinstance(irnd, int) else '%4s') % irnd
            
            log_lines.append('%25s | %20s | %s | %s' % (fname, meth, srnd, vdict))
    
        rnd_broken = max_rnd_broken_meth[meth]
        rnd_comp = max_rnd_meth[meth]
        rnd_comp_min = min_rnd_meth[meth]
        
        # Compute also x.key for block functions
        if erec.ftype == FuncInfo.CIPHER and is_below_1000 and not is_col_type:
            meth_broken = max_rnd_broken_meth_100[meth]
            orig_range = list(range(meth_broken - 1, meth_broken + 2))
            recomp_list = [x for x in orig_range if x > 0 and x <= max_rounds_cur]
            
            nmeth = f'{meth_parts[0]}.key:{":".join(meth_parts[1:])}'
            compute_cols[ftypename][nmeth] = recomp_list
            
            # Another generation methods
            #if erec.stype not in (FuncInfo.STREAM, FuncInfo.MPC):
            #    nmeth = f'{meth_parts[0]}.key..inp.rnd:{":".join(meth_parts[1:])}'
            #    compute_cols[ftypename][nmeth] = recomp_list
        
        # recomp list: start with new rounds only (max computed), upper bound - either max rounds or broken + limit 
        if fname in funcs_no_rounds:
            continue
        
        # 10, 100 MB comp lists
        if True or meth_size < h1000MB:
            recomp_list = list(range(max(rnd_comp + 1, rnd_broken + 1), min(max_rounds_cur + 1, max(rnd_comp + 1, rnd_broken + 1 + exids_recomp_margin)))) 
            if rnd_broken == 0 and rnd_comp > 0:
                recomp_list = list(range(rnd_comp_min - 2, rnd_comp_min)) + recomp_list  # 2 rounds down
            exids_recompute[ftypename][meth] = [cexids, [x for x in sorted(list(set(recomp_list))) if x > 0 and x <= max_rounds_cur]]

        # 1000MB, if detected on the max, add one more; if not detected on minimal, add less more.
        if meth_size >= h1000MB:
            meth_100 = ':'.join(meth_parts[:-1]) + (':%s' % h100MB)
            recomp_list = []
            if rnd_broken == 0 and rnd_comp > 0:  # not detected on minimal, add X less
                recomp_list += list(range(rnd_comp_min - 2, rnd_comp_min))
            if rnd_broken == rnd_comp:  # detected on max, add X more
                recomp_list += list(range(rnd_comp + 1, rnd_comp + 3))
            
            # Add highest detected in 100MB, -1 also for confirmation, +1 for overreach
            if meth_100 in max_rnd_broken_meth and meth_100 in max_rnd_meth:
                br100 = max_rnd_broken_meth[meth_100]
                recomp_list += list(range(br100 - 1, br100 + 2))

            # exids_recompute[ftypename][meth] = (cexids, [x for x in sorted(list(set(recomp_list))) if x > 0 and x <= max_rounds_cur])
            exids_recompute[ftypename][meth][1] = sorted(list(set(exids_recompute[ftypename][meth][1] + [x for x in sorted(list(set(recomp_list))) if x > 0 and x <= max_rounds_cur])))
        
        # recomp 1000MB for upper levels, new sizes
        if meth_size < h100MB:
            continue
        
        meth_1000 = ':'.join(meth_parts[:-1]) + (':%s' % h1000MB)
        recomp_list_1000MB = list(range(max(rnd_broken + 1, rnd_broken + 1), min(max_rounds_cur + 1, max(rnd_broken + 1, rnd_broken + 1 + exids_recomp_margin_1000MB)))) 
        exids_recompute_1000MB[ftypename][meth_1000] = [cexids, recomp_list_1000MB]
        
        if meth_size >= h1000MB:
            exids_computed_1000MB[ftypename][meth] = max(exids_computed_1000MB[ftypename][meth], rnd_comp)
    
    # add zero stream, base on ctr estimate
    if erec.ftype == FuncInfo.CIPHER and erec.stype == FuncInfo.STREAM:
        meth_base_sizes = set([int(x.split(':')[-1]) for x in exids_recompute[ftypename].keys() if '.key' not in x])
        for csize in sorted(list(meth_base_sizes)):
            ctr_mth = f'ctr:{csize}'
            zro_mth = f'zero:{csize}'

            if zro_mth not in exids_recompute[ftypename]:
                if ctr_mth in comp_rounds_meth:
                    b1 = exids_recompute[ftypename][ctr_mth][1] if ctr_mth in exids_recompute[ftypename] else []
                    exids_recompute[ftypename][zro_mth] = [[], sorted(list(set(b1 + comp_rounds_meth[ctr_mth])))]
                elif ctr_mth in exids_recompute[ftypename]:
                    exids_recompute[ftypename][zro_mth] = list(exids_recompute[ftypename][ctr_mth])
                else:
                    logger.info(f'! Cannot add zero input for {ftypename}')

    # .key generator: if rnd.key not present, base on sac estimate
    col_meth_sizes = set([int(x.split(':')[-1]) for x in compute_cols[ftypename].keys()])
    for csize in sorted(list(col_meth_sizes)):
        rnd_mth = f'rnd.key:{csize}'
        sac_mth = f'sac.key:{csize}'
        ctr_mth = f'ctr.key:{csize}'

        if rnd_mth not in compute_cols[ftypename]:
            if sac_mth in comp_rounds_meth:
                compute_cols[ftypename][rnd_mth] = comp_rounds_meth[sac_mth]
            elif sac_mth in compute_cols[ftypename]:
                compute_cols[ftypename][rnd_mth] = list(compute_cols[ftypename][sac_mth])
            elif ctr_mth in compute_cols[ftypename]:
                compute_cols[ftypename][rnd_mth] = list(compute_cols[ftypename][ctr_mth])
            else:
                logger.info(f'! Cannot add rnd.key for {ftypename}')
    
    # Adapt exids_recompute_1000MB, remove already computed rounds
    for cmeth in exids_computed_1000MB[ftypename].keys():
        max_rnd = exids_computed_1000MB[ftypename][cmeth]
        if cmeth not in exids_recompute_1000MB[ftypename]:
            continue
        if exids_recompute_1000MB[ftypename][cmeth] is None:
            continue
        exids_recompute_1000MB[ftypename][cmeth][1] = [x for x in exids_recompute_1000MB[ftypename][cmeth][1] if x > max_rnd]
    
    for cmeth in computed_cols[ftypename].keys():
        compute_cols[ftypename][cmeth] = [x for x in compute_cols[ftypename][cmeth] if x not in computed_cols[ftypename][cmeth]]
    
    # comp cleanup
    for cmeth in list(exids_computed_1000MB[ftypename].keys()):
        if exids_recompute_1000MB[ftypename][cmeth] is None or len(exids_recompute_1000MB[ftypename][cmeth][1]) == 0:
            del exids_recompute_1000MB[ftypename][cmeth]
    for cmeth in list(exids_recompute[ftypename].keys()):
        if exids_recompute[ftypename][cmeth] is None or len(exids_recompute[ftypename][cmeth][1]) == 0:
            del exids_recompute[ftypename][cmeth]
    
    if max_rnd_broken is None and crypt_rounds_cur is None and max_rounds_cur is None:
        print('Skipping null %s' % fname)
        continue
    
    nfname = fname
    nfname = nfname.replace('ROAD-RUNNER-K80', 'R.RUNNER.K80')
    nfname = nfname.replace('ROAD-RUNNER-K128', 'R.RUNNER.K128')
    nfname = nfname.replace('RECTANGLE-K80', 'RECT.K80')
    nfname = nfname.replace('RECTANGLE-K128', 'RECT.K128')
    
    nfname = nfname.replace('testu01-uxorshift', 'U01.XorShift')
    nfname = nfname.replace('testu01-umrg', 'U01.UMRG')
    nfname = nfname.replace('testu01-ulcg', 'U01.ULCG')
    nfname = nfname.replace('std_subtract_with_carry', 'Std.SubCarry')
    nfname = nfname.replace('std_mersenne_twister', 'Std.MTwister')
    nfname = nfname.replace('std_lcg', 'Std.LCG')
    
    nfname = nfname.replace('mimc_hash-', 'MiMC.')
    nfname = nfname.replace('gmimc-', 'GMiMC.')
    nfname = nfname.replace('lowmc-s', 'LowMC.S')
    nfname = nfname.replace('Vision_', 'Vision.')
    nfname = nfname.replace('Starkad_', 'Starkad.')
    nfname = nfname.replace('Rescue_', 'Rescue.')
    nfname = nfname.replace('Poseidon_S128_BLS12_138', 'Poseidon.BLS12')
    nfname = nfname.replace('Poseidon_', 'Poseidon.')
    
    cline = '%s & \\boxSecMarginXX{%s}{%s}{%s}' % (nfname, max_rnd_broken or 0, max_rounds_cur or 1, crypt_rounds_cur or 0)
    table_lines.append((cline, erec))
    
    #if len(table_lines)> 10: break

    
print("\n".join(log_lines))

cols = 3
per_col = math.ceil(len(table_lines)/3.0)
sorter = lambda x: (x[1].ftype, x[1].stype, natural_sort_key(x[1].fname))
table_lines.sort(key=sorter)
ntable_lines = []

print("\n\nTable lines: ", len(table_lines), ", per col:", per_col)
# for k, g in itertools.groupby(table_lines, lambda x: (x[1].ftype)):

old_type=None
for e in table_lines:
    if (e[1].ftype, e[1].stype) != old_type:
        old_type = (e[1].ftype, e[1].stype)
        if old_type[0] == FuncInfo.HASH:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Hash functions}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 0:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Block ciphers}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 2:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Stream ciphers}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 4:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{MPC ciphers}}')
        elif old_type[0] == FuncInfo.PRNG:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{PRNGs}}')
        
    ntable_lines.append(e[0])

print(len(ntable_lines))
per_col = math.ceil(len(ntable_lines)/3.0)
arcols = [ntable_lines[i*per_col: (i+1)*per_col] for i in range(cols)]

for x in range(per_col):
    tup = arcols[0][x] if x < len(arcols[0]) else '& ', arcols[1][x] if x < len(arcols[1]) else '& ', arcols[2][x] if x < len(arcols[2]) else '& '
    print("& ".join(tup) + "\\\\")

#for x in range(per_col):
#    l1 =    
#print("\n".join(ntable_lines))

# Specs for adding new rounds to compute
print('-----\n\n\n Compute more rounds for 10/100:\n', json.dumps(exids_recompute))#, indent=2))    
print('-----\n\n\n Compute more rounds for 1000:\n', json.dumps(exids_recompute_1000MB))#, indent=2))         
print('-----\n\n\n Compute cols:\n', json.dumps(compute_cols))#, indent=2))

In [56]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))